In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import warnings
warnings.filterwarnings('ignore')
from ast import literal_eval
import plotly.express as px
import nltk
import re

Trabalhar com Dados Numericos

In [3]:
PATH = '/content/drive/My Drive/TCCDataScience/datasets/ml-25m/'
movies = pd.read_csv(PATH+'movies.csv')
#genome_scores = pd.read_csv(PATH+'genome-scores.csv')
tags = pd.read_csv(PATH+'tags.csv')
#genome_tags = pd.read_csv(PATH+'genome-tags.csv')
#Use ratings data to downsample tags data to only movies with ratings 
tmdb = pd.read_csv('/content/drive/My Drive/TCCDataScience/datasets/tmdb_5000_movies.csv')
# ratings = pd.read_csv(PATH+'ratings.csv')
links = pd.read_csv(PATH+'links.csv')
links.drop('imdbId', inplace=True, axis=1) 
links.dropna(inplace=True, axis=0)
links['tmdbId']=links['tmdbId'].astype('int')

In [4]:
links.head() 
links.shape

(62316, 2)

In [5]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [6]:
tmdb = tmdb.drop('genres', axis=1) 
tmdb = tmdb.drop('title', axis=1)
tmdb = tmdb.drop('status', axis=1)
tmdb = tmdb.drop('spoken_languages', axis=1)
tmdb = tmdb.drop('release_date', axis=1)
tmdb = tmdb.drop('production_companies', axis=1)
tmdb = tmdb.drop('production_countries',  axis=1)
tmdb = tmdb.drop('overview',  axis=1)
tmdb = tmdb.drop('keywords',axis=1)
tmdb = tmdb.drop('homepage', axis=1)
tmdb = tmdb.drop('original_language',axis=1)
tmdb = tmdb.drop('original_title', axis=1)
tmdb.rename(columns={"id": "tmdbId"}, inplace=True)
#  0   budget                4803 non-null   int64  
#  8   popularity            4803 non-null   float64 
#  12  revenue               4803 non-null   int64  
#  13  runtime               4801 non-null   float64 
#  18  vote_average          4803 non-null   float64
#  19  vote_count            4803 non-null   int64   
tmdb.head(3)

,budget,tmdbId,popularity,revenue,runtime,tagline,vote_average,vote_count
0,237000000,19995,150.437577,2787965087,162.0,Enter the World of Pandora.,7.2,11800
1,300000000,285,139.082615,961000000,169.0,"At the end of the world, the adventure begins.",6.9,4500
2,245000000,206647,107.376788,880674609,148.0,A Plan No One Escapes,6.3,4466


In [7]:
movies = pd.merge(movies, links, on='movieId',how='inner')

In [8]:
movies.shape

(62316, 4)

In [9]:
movies.head(3)

,movieId,title,genres,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862
1,2,Jumanji (1995),Adventure|Children|Fantasy,8844
2,3,Grumpier Old Men (1995),Comedy|Romance,15602


In [10]:
movies = pd.merge(movies, tmdb, on='tmdbId',how='inner')
movies.head(3)

,movieId,title,genres,tmdbId,budget,popularity,revenue,runtime,tagline,vote_average,vote_count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862,30000000,73.640445,373554033,81.0,NaN,7.7,5269
1,10,GoldenEye (1995),Action|Adventure|Thriller,710,58000000,59.824565,352194034,130.0,No limits. No fears. No substitutes.,6.6,1174
2,11,"American President, The (1995)",Comedy|Drama|Romance,9087,62000000,11.056763,107879496,106.0,Why can't the most powerful man in the world h...,6.5,195


In [11]:
movies.set_index('movieId',inplace=True)
movies.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4602 entries, 1 to 206731
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         4602 non-null   object 
 1   genres        4602 non-null   object 
 2   tmdbId        4602 non-null   int64  
 3   budget        4602 non-null   int64  
 4   popularity    4602 non-null   float64
 5   revenue       4602 non-null   int64  
 6   runtime       4602 non-null   float64
 7   tagline       3875 non-null   object 
 8   vote_average  4602 non-null   float64
 9   vote_count    4602 non-null   int64  
dtypes: float64(3), int64(4), object(3)
memory usage: 395.5+ KB


In [12]:
movies_features = pd.DataFrame(movies, columns= ["budget","popularity","revenue",
                                                    "runtime", "vote_count"])
movies_target =  pd.DataFrame(movies, columns= [ "vote_average" ])

In [13]:
# baseline - what's the score of all the features? Answer: Not very good. we can do better.
from sklearn.model_selection import cross_val_score, ShuffleSplit, train_test_split
from sklearn.linear_model import LinearRegression
scores = cross_val_score(LinearRegression(), movies_features, movies_target, scoring='r2') # mean_squared_error
print("Linear Regression Accuracy all Features: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Linear Regression Accuracy all Features: 0.17 (+/- 0.30)


Step 2: Select Best Features
----------------------------

We are going to use three methods to select the best features for predicting "vote_average"
The three methods for getting scores for features are: Pearson, SelectKBest and RandomForestRegressor


In [14]:
movies.corr(method='pearson') # We are getting similar results for - pearson, kendall, spearman
# Top 3 Features:
#budget
#popularity
#vote_count

,tmdbId,budget,popularity,revenue,runtime,vote_average,vote_count
tmdbId,1.000000,-0.061363,0.074096,-0.028180,-0.111723,-0.177082,0.029484
budget,-0.061363,1.000000,0.497393,0.727293,0.269796,0.053965,0.587989
popularity,0.074096,0.497393,1.000000,0.641059,0.222443,0.277669,0.775065
revenue,-0.028180,0.727293,0.641059,1.000000,0.256801,0.197099,0.779950
runtime,-0.111723,0.269796,0.222443,0.256801,1.000000,0.373175,0.277266
vote_average,-0.177082,0.053965,0.277669,0.197099,0.373175,1.000000,0.333648
vote_count,0.029484,0.587989,0.775065,0.779950,0.277266,0.333648,1.000000


Step 2(b) : SelectKBest
-----------------------

In [15]:
from sklearn.feature_selection import SelectKBest, f_regression
movies_best2 = SelectKBest(f_regression, k=2).fit_transform(movies_features, movies_target) 

In [16]:
movies_best2

array([[8.100e+01, 5.269e+03],
       [1.300e+02, 1.174e+03],
       [1.060e+02, 1.950e+02],
       ...,
       [8.700e+01, 1.000e+01],
       [6.800e+01, 2.000e+00],
       [9.000e+01, 1.400e+01]])

Step 1(c): Renaming
-------------------

- Features --> IMDB_Features_df
- Label --> IMDB_Label_df
- IMDB_Features_df + IMDB_Label_df --> IMDB_df

In [17]:
targetMatrix = movies_target['vote_average'].to_numpy()
featuresMatrix = movies_features.to_numpy() #This Matrix will be used in finding best features using Random Forest

In [52]:
movies_features.head(3)
featuresMatrix

array([[3.00000000e+07, 7.36404450e+01, 3.73554033e+08, 8.10000000e+01,
        5.26900000e+03],
       [5.80000000e+07, 5.98245650e+01, 3.52194034e+08, 1.30000000e+02,
        1.17400000e+03],
       [6.20000000e+07, 1.10567630e+01, 1.07879496e+08, 1.06000000e+02,
        1.95000000e+02],
       ...,
       [3.00000000e+06, 1.27910600e+00, 1.20000000e+06, 8.70000000e+01,
        1.00000000e+01],
       [0.00000000e+00, 1.86401000e-01, 0.00000000e+00, 6.80000000e+01,
        2.00000000e+00],
       [4.50000000e+05, 1.07732100e+00, 0.00000000e+00, 9.00000000e+01,
        1.40000000e+01]])

In [18]:
print ("Number of observations: {}\nNumber of features {}".\
    format(movies.shape[0], movies.shape[1]))

Number of observations: 4602
Number of features 10


In [19]:
movies.head(3)

,title,genres,tmdbId,budget,popularity,revenue,runtime,tagline,vote_average,vote_count
movieId,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862,30000000,73.640445,373554033,81.0,NaN,7.7,5269
10,GoldenEye (1995),Action|Adventure|Thriller,710,58000000,59.824565,352194034,130.0,No limits. No fears. No substitutes.,6.6,1174
11,"American President, The (1995)",Comedy|Drama|Romance,9087,62000000,11.056763,107879496,106.0,Why can't the most powerful man in the world h...,6.5,195


Finding out the Accuracy of Linear Regression for all features

> Bloco com recuo


--------------------------------------------------------------

In [20]:
from sklearn.model_selection import cross_val_score, ShuffleSplit, train_test_split
from sklearn.linear_model import LinearRegression

# baseline - what's the score of all the features? Answer: Not very good. we can do better.
scores = cross_val_score(LinearRegression(), featuresMatrix, targetMatrix, scoring='r2') # mean_squared_error
print("Linear Regression Accuracy all Features: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#We are going to further select best features and then try to find out the accuracy again

Linear Regression Accuracy all Features: 0.17 (+/- 0.30)


Step 2(a) : Pearson
-------------------

Finding out features which are correlated with the label (imdb_score)

In [21]:
movies.corr(method='pearson') # We are getting similar results for - pearson, kendall, spearman
# Top 3 Features: pairplot for continuous values
#budget
#popularity
#vote_count

,tmdbId,budget,popularity,revenue,runtime,vote_average,vote_count
tmdbId,1.000000,-0.061363,0.074096,-0.028180,-0.111723,-0.177082,0.029484
budget,-0.061363,1.000000,0.497393,0.727293,0.269796,0.053965,0.587989
popularity,0.074096,0.497393,1.000000,0.641059,0.222443,0.277669,0.775065
revenue,-0.028180,0.727293,0.641059,1.000000,0.256801,0.197099,0.779950
runtime,-0.111723,0.269796,0.222443,0.256801,1.000000,0.373175,0.277266
vote_average,-0.177082,0.053965,0.277669,0.197099,0.373175,1.000000,0.333648
vote_count,0.029484,0.587989,0.775065,0.779950,0.277266,0.333648,1.000000


In [51]:
from sklearn import model_selection
xtrain, xval, ytrain, yval = model_selection.train_test_split(featuresMatrix, targetMatrix, test_size=0.2, random_state=512)
RF_set= xval[['budget',	'popularity',	'revenue',	'runtime',	'vote_count']]
RF_set

IndexError: ignored

Aplicação da Regressão Logistica

In [30]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [37]:
lr = LogisticRegression()
clf = OneVsRestClassifier(LogisticRegression()).fit(xtrain, ytrain)

 

ValueError: ignored

In [34]:
clf.fit(featuresMatrix, targetMatrix)

ValueError: ignored

Step 2(b) : SelectKBest
-----------------------

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
# Return the 2 features with highest variance 
IMDB_best2 = SelectKBest(f_regression, k=2).fit_transform( featuresMatrix, targetMatrix) 

In [ ]:
IMDB_best2
# runtime	
# vote_count

array([[8.100e+01, 5.269e+03],
       [1.300e+02, 1.174e+03],
       [1.060e+02, 1.950e+02],
       ...,
       [8.700e+01, 1.000e+01],
       [6.800e+01, 2.000e+00],
       [9.000e+01, 1.400e+01]])

Step 2(c) : Random Forest Regressor
-----------------------------------

In [22]:
names1=["budget","popularity","revenue","runtime", "vote_count"] 

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit

Step 3 : Splitting the dataset into Train, Test, Validation Dataset
-------------------------------------------------------------------

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train_validation, X_test, y_train_validation, y_test = train_test_split(featuresMatrix, targetMatrix, 
                                                                          test_size = 0.1, 
                                                                          random_state = 0)

In [40]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train_validation, 
                                                                y_train_validation, 
                                                                test_size = 0.1, 
                                                                random_state = 0)


Train:70% , Test: 20% , Validation: 10%

In [41]:
X_train_validation1, X_test1, y_train_validation1, y_test1 = train_test_split(featuresMatrix, targetMatrix, 
                                                                          test_size = 0.2, 
                                                                          random_state = 0)

In [42]:
X_train1, X_validation1, y_train1, y_validation1 = train_test_split(X_train_validation1, 
                                                                y_train_validation1, 
                                                                test_size = 0.1, 
                                                                random_state = 0)

Train:50% , Test: 40%

In [43]:
X_train_validation2, X_test2, y_train_validation2, y_test2 = train_test_split(featuresMatrix, targetMatrix,
                                                                          test_size = 0.4, 
                                                                          random_state = 0)

In [44]:

X_train2, X_validation2, y_train2, y_validation2 = train_test_split(X_train_validation2, 
                                                                y_train_validation2, 
                                                                test_size = 0.15, 
                                                                random_state = 0)

In [47]:
featuresMatrix

array([[3.00000000e+07, 7.36404450e+01, 3.73554033e+08, 8.10000000e+01,
        5.26900000e+03],
       [5.80000000e+07, 5.98245650e+01, 3.52194034e+08, 1.30000000e+02,
        1.17400000e+03],
       [6.20000000e+07, 1.10567630e+01, 1.07879496e+08, 1.06000000e+02,
        1.95000000e+02],
       ...,
       [3.00000000e+06, 1.27910600e+00, 1.20000000e+06, 8.70000000e+01,
        1.00000000e+01],
       [0.00000000e+00, 1.86401000e-01, 0.00000000e+00, 6.80000000e+01,
        2.00000000e+00],
       [4.50000000e+05, 1.07732100e+00, 0.00000000e+00, 9.00000000e+01,
        1.40000000e+01]])

In [46]:
X_validation2 

SyntaxError: ignored

Train : 30 % , Test : 60%

In [ ]:
X_train_validation3, X_test3, y_train_validation3, y_test3 = train_test_split(IMDB_Features_df, 
                                                                          IMDB_Label_df, 
                                                                          test_size = 0.6, 
                                                                          random_state = 0)

In [ ]:

X_train3, X_validation3, y_train3, y_validation3 = train_test_split(X_train_validation3, 
                                                                y_train_validation3, 
                                                                test_size = 0.2, 
                                                                random_state = 0)

Train : 15 %, Test : 80%

In [ ]:
X_train_validation4, X_test4, y_train_validation4, y_test4 = train_test_split(IMDB_Features_df, 
                                                                          IMDB_Label_df, 
                                                                          test_size = 0.8, 
                                                                          random_state = 0)

In [ ]:
X_train_validation4, X_test4, y_train_validation4, y_test4 = train_test_split(IMDB_Features_df, 
                                                                          IMDB_Label_df, 
                                                                          test_size = 0.8, 
                                                                          random_state = 0)